In [ ]:
"""
Create fishnet
"""

from gasp3.gt.sample import create_fishnet

inShp = r'D:\gisbrary\admin_lmt\CAOP2017\pt_lmt.shp'

outShp = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'

create_fishnet(inShp, outShp, width=20000, height=20000)

In [ ]:
"""
Count vizinhos cell in fishnet
"""

fishnet = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'

res_x = 20000
res_y = 20000

from gasp3.fm import tbl_to_obj
from gasp3.g.gop.prox  import df_buffer_extent

fish_df = tbl_to_obj(fishnet)

bf_df = df_buffer_extent(grid_df, epsg, cell_width, mantainOriginalGeom=None)

In [ ]:
print